In [1]:
# This cell is only used to suppress some distracting output messages
import warnings

warnings.filterwarnings("ignore", category=UserWarning)


In [2]:
import matplotlib.pyplot as plt
from cartopy import crs
from cartopy import feature as cfeature
from rich import print
import pandas as pd
import xarray as xr
from pathlib import Path
from datetime import datetime
from ipywidgets import IntProgress
from IPython.display import display
import fiona
import shapely.geometry
from pyproj import Geod

import ewatercycle
import ewatercycle.forcing
import ewatercycle.models
import ewatercycle.parameter_sets

In [3]:
# Chatly_station_latitude = 42.34332908492399  # Amu Darya near Chatly
# Chatly_station_longitude = 59.627516175820965 

Chatly_station_latitude = 42.34332908492399-0.5 # Amu Darya near Chatly
Chatly_station_longitude = 59.627516175820965+0.9    # Through trial and error, to get modelled discharge closer to observed

Kerki_station_latitude = 37.8396310038444 #Amu Darya near Kerki
Kerki_station_longitude = 65.23703868931334

#Tyumen_station_latitude = 44.01445789449254 # Syr Darya near Tyumen, google maps
#Tyumen_station_longitude = 67.02866313732494

#Tyumen_station_latitude = 44.05 # Syr Darya near Tyumen, GRDC coords
#Tyumen_station_longitude = 67.05

Tyumen_station_latitude = 44.05-0.1 # Syr Darya near Tyumen, through trial and error
Tyumen_station_longitude = 67.05

Kazalinsk_station_latitude = 45.739988222442456, #Syr Darya near Kazalinsk
Kazalinsk_station_longitude = 62.115993992559744

In [4]:
pcr_glob_directory = Path("/data/shared/parameter-sets/pcrglobwb_global")  #GlobalOption uit .ini


#prepared_PCRGlob_forcing = Path.cwd()/"forcing_1010" / "AralSeaBasin" / "pcrglobwb" / "work/diagnostic/script" ##MeteoOptions uit .ini
prepared_PCRGlob_forcing = "/home/avandervee3/MSc_AralSea/book/thesis_projects/MSc/2025_Q1_AndreVanDerVeen_CEG/work_in_progress/Test_Aral/forcing_1010/AralSeaBasin/pcrglobwb/work/diagnostic/script"

In [17]:

parameter_set = ewatercycle.parameter_sets.ParameterSet(
    name="custom_parameter_set",
    directory=pcr_glob_directory,
    config= Path.cwd() / "pcrglobwb_aral_basin_ERA5_05min.ini",
    target_model="pcrglobwb",
    supported_model_versions={"setters_update"},
)



In [18]:
# set start and end date of the experiment, overwrites .ini settings
#experiment_start_date = "1987-01-01T00:00:00Z"
#experiment_end_date = "1992-12-31T00:00:00Z" #was 1995-12-31T00:00:00Z, shorter for testing
#experiment_end_date = "1990-02-28T00:00:00Z" #2 months, for testing
#experiment_end_date = "1987-07-31T00:00:00Z" #2 months, for testing

experiment_start_date="2010-06-01T00:00:00Z"
experiment_end_date="2010-07-31T00:00:00Z"

In [19]:
forcing = ewatercycle.forcing.sources["PCRGlobWBForcing"].load(
    directory=prepared_PCRGlob_forcing,
)




print(forcing)

PCRGlobWBForcing(
    start_time='2010-09-01T00:00:00Z',
    end_time='2010-12-31T00:00:00Z',
    directory=PosixPath('/home/avandervee3/MSc_AralSea/book/thesis_projects/MSc/2025_Q1_AndreVanDerVeen_CEG/work_in
_progress/Test_Aral/forcing_1010/AralSeaBasin/pcrglobwb/work/diagnostic/script'),
    shape=PosixPath('/home/avandervee3/MSc_AralSea/book/thesis_projects/MSc/2025_Q1_AndreVanDerVeen_CEG/work_in_pro
gress/Test_Aral/forcing_1010/AralSeaBasin/pcrglobwb/work/diagnostic/script/AralSeaBasin.shp'),
    filenames={},
    precipitationNC='pcrglobwb_OBS6_ERA5_reanaly_1_day_pr_2010-2010_AralSeaBasin.nc',
    temperatureNC='pcrglobwb_OBS6_ERA5_reanaly_1_day_tas_2010-2010_AralSeaBasin.nc'
)

In [20]:
from ewatercycle_pcrglobwb.model import ContainerImage

# parameter_set = ewatercycle.parameter_sets.ParameterSet(
#     name="custom_parameter_set",
#     directory=pcr_glob_directory,
#     config= Path.cwd() / "pcrglobwb_aral_basin_ERA5_05min.ini",
#     target_model="pcrglobwb",
#     supported_model_versions={"1.0"},
# )



my_image = ContainerImage("/home/avandervee3/pcrglobwb-grpc4bmi:setters_update.sif")

reference = ewatercycle.models.PCRGlobWB(
    parameter_set=parameter_set,
    forcing=forcing,
    bmi_image=my_image
)



ValidationError: 1 validation error for PCRGlobWB
  Value error, Parameter set 'custom_parameter_set' not compatible with this model version.
Model version: update. Compatible versions: {'setters_update'} [type=value_error, input_value={'parameter_set': Paramet...bmi:setters_update.sif'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [ ]:
print(reference.parameters)

In [ ]:
from ewatercycle_pcrglobwb.model import ContainerImage
from pathlib import Path

my_image = ContainerImage("/home/avandervee3/pcrglobwb_custom.sif")
print("Container model version:", my_image.version)

In [ ]:
## hier wordt de nieuwe map gemaakt, verschijnt nu in dezelfde map als notebook
## bevat hier alleen nog .ini file

reference_config, reference_dir = reference.setup(
    start_time = experiment_start_date,
    end_time = experiment_end_date,
    max_spinups_in_years=0
)
reference_config, reference_dir


In [ ]:
reference.logs()

In [ ]:
print(reference.parameters)

refence_para = reference.parameters

# Convert ISO 8601 strings to datetime objects
start_time = datetime.strptime(experiment_start_date, '%Y-%m-%dT%H:%M:%SZ')
end_time = datetime.strptime(experiment_end_date, '%Y-%m-%dT%H:%M:%SZ')

# Calculate the number of days for the progression bar
delta = end_time - start_time
number_of_days = delta.days
print(f"Number of days to model: {number_of_days}")

In [ ]:
#pcrglob.initialize(cfg_file)
reference.initialize(reference_config)

In [ ]:
time = pd.date_range(reference.start_time_as_isostr, reference.end_time_as_isostr)
# timeseries = pd.DataFrame(
#     index=pd.Index(time, name="time"), columns=["reference", "experiment1", "experiment2"]
# )
# timeseries.head()

In [ ]:
Stations_timeseries = pd.DataFrame(
    index=pd.Index(time, name="time"), columns=["Chatly", "Kerki", "Tyumen", "Kazalinsk"]
)
Stations_timeseries.head()

Stations_timeseries_experiment = Stations_timeseries.copy();

In [ ]:
# Progress bar, since this can take a while
f = IntProgress(min=0, max=number_of_days) # instantiate the bar
display(f) # display the bar

while reference.time < reference.end_time:

    reference.update()

    # Track discharge at station location
    discharge_at_Chatly = reference.get_value_at_coords(
        "discharge", lat=[Chatly_station_latitude], lon=[Chatly_station_longitude]
    )
    time = reference.time_as_isostr
    Stations_timeseries.loc[time, "Chatly"] = discharge_at_Chatly[0]

     # Track discharge at station location Kerki
    discharge_at_Kerki = reference.get_value_at_coords(
        "discharge", lat=[Kerki_station_latitude], lon=[Kerki_station_longitude]
    )
    time = reference.time_as_isostr
    Stations_timeseries.loc[time, "Kerki"] = discharge_at_Kerki[0]

    # Track discharge at station Tyumen
    discharge_at_Tyumen = reference.get_value_at_coords(
        "discharge", lat=[Tyumen_station_latitude], lon=[Tyumen_station_longitude]
    )
    time = reference.time_as_isostr
    Stations_timeseries.loc[time, "Tyumen"] = discharge_at_Tyumen[0]

    # Track discharge at station location Karalinsk
    discharge_at_Kazalinsk = reference.get_value_at_coords(
        "discharge", lat=[Kazalinsk_station_latitude], lon=[Kazalinsk_station_longitude]
    )
    time = reference.time_as_isostr
    Stations_timeseries.loc[time, "Kazalinsk"] = discharge_at_Kazalinsk[0]


    f.value += 1

print("Model run finished!")

In [ ]:
Stations_timeseries.head()

In [ ]:
# Use matplotlib to make the figure slightly nicer
fig = plt.figure(figsize=(10,8),dpi=120)
#plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection=crs.PlateCarree())

# Plotting the model field is a one-liner
reference.get_value_as_xarray("discharge").plot(ax=ax, cmap="GnBu")

# Also plot the station location
ax.scatter(Chatly_station_longitude+1, Chatly_station_latitude-0.5, s=25, c="r", label = "Chatly")
ax.scatter(Kerki_station_longitude, Kerki_station_latitude, s=25, c="r", label = "Kerki")
ax.scatter(Tyumen_station_longitude, Tyumen_station_latitude, s=25, c="r", label = "Tyumen")
ax.scatter(Kazalinsk_station_longitude, Kazalinsk_station_latitude, s=25, c="r", label= "Kazalinsk")

# Overlay ocean and coastines
ax.add_feature(cfeature.OCEAN, zorder=2)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.LAKES, alpha=0.8)
#ax.add_feature(cfeature.RIVERS)
#ax.add_feature(cfeature.RIVERS, zorder=2, color="k")
ax.coastlines(zorder=3)

In [ ]:
Stations_timeseries.plot(label = Stations_timeseries.columns)

In [ ]:
soil_moisture = reference.get_value_as_xarray("upper_soil_saturation_degree")

# Copy the field and manually overwrite a random part of the domain
irrigated_soil_moisture = soil_moisture.copy()
irrigated_soil_moisture[11:141, 178:278] = 1 

In [ ]:
fig = plt.figure(figsize=(10, 5), dpi=120)
left_axes = fig.add_subplot(121, projection=crs.PlateCarree())
right_axes = fig.add_subplot(122, projection=crs.PlateCarree())

soil_moisture.plot(ax=left_axes, cmap="GnBu", vmin=0.3, vmax=1)
irrigated_soil_moisture.plot(ax=right_axes, cmap="GnBu", vmin=0.3, vmax=1)

# Decoration
left_axes.set_title("Reference")
right_axes.set_title("Irrigated patch")

for axes in [left_axes, right_axes]:
    axes.add_feature(cfeature.OCEAN, zorder=2)
    axes.add_feature(cfeature.RIVERS, zorder=2, color="k")
    axes.coastlines(zorder=3)
    # Also plot the station location
    axes.scatter(Chatly_station_longitude, Chatly_station_latitude, s=25, c="r", label = "Chatly")
    axes.scatter(Kerki_station_longitude, Kerki_station_latitude, s=25, c="r", label = "Kerki")
    axes.scatter(Tyumen_station_longitude, Tyumen_station_latitude, s=25, c="r", label = "Tyumen")
    axes.scatter(Kazalinsk_station_longitude, Kazalinsk_station_latitude, s=25, c="r", label= "Kazalinsk")
    axes.scatter(70,36, s=25, c="yellow", label = "measurements", marker='x')



In [ ]:
experiment = ewatercycle.models.PCRGlobWB(
    parameter_set=parameter_set,
    forcing = forcing
)

print(experiment)

In [ ]:
#Creates new folder, only contains .ini file

experiment_config, experiment_dir = experiment.setup(
    start_time = experiment_start_date,
    end_time = experiment_end_date,
    max_spinups_in_years=0
)



In [ ]:
print(experiment.parameters)

expermient_para = experiment.parameters

# Convert ISO 8601 strings to datetime objects
start_time = datetime.strptime(experiment_start_date, '%Y-%m-%dT%H:%M:%SZ')
end_time = datetime.strptime(experiment_end_date, '%Y-%m-%dT%H:%M:%SZ')

# Calculate the number of days for the progression bar
delta = end_time - start_time
number_of_days = delta.days
print(f"Number of days to model: {number_of_days}")

In [ ]:
experiment.initialize(experiment_config)

In [ ]:
# Progress bar, since this can take a while
g = IntProgress(min=0, max=number_of_days) # instantiate the bar
display(g) # display the bar
values_pre_before = []
values_pre_after = []
values_post_before = []
values_post_after = []
#values_after = []


experiment.update()

while experiment.time < experiment.end_time:
    # Check before overwriting
    soil_check_before = experiment.get_value_at_coords(
        "upper_soil_saturation_degree", lat=[36], lon=[70]
    )

    values_pre_before.append(soil_check_before[0])

    # Overwrite before update
    soil_moisture = experiment.get_value_as_xarray("upper_soil_saturation_degree")
    soil_moisture[11:141, 178:278] = 1
    experiment.set_value("upper_soil_saturation_degree", soil_moisture.values.flatten())

    # Check after overwriting
    soil_check_after = experiment.get_value_at_coords(
        "upper_soil_saturation_degree", lat=[36], lon=[70]
    )
    values_pre_after.append(soil_check_after[0])




    # update the model by one time step
    experiment.update()
    

    # Check s before overwriting
    soil_check_before = experiment.get_value_at_coords(
        "upper_soil_saturation_degree", lat=[36], lon=[70]
    )
    values_post_before.append(soil_check_before[0])

    # Overwrite after update
    soil_moisture = experiment.get_value_as_xarray(
        "upper_soil_saturation_degree",
    )
    soil_moisture[11:141, 178:278] = 1
    experiment.set_value("upper_soil_saturation_degree", soil_moisture.values.flatten())

    # Check after overwriting
    soil_check_after = experiment.get_value_at_coords(
        "upper_soil_saturation_degree", lat=[36], lon=[70]
    )
    values_post_after.append(soil_check_after[0])



    

    # Track discharge at station location
    discharge_at_Chatly = experiment.get_value_at_coords(
        "discharge", lat=[Chatly_station_latitude], lon=[Chatly_station_longitude]
    )
    time = experiment.time_as_isostr
    Stations_timeseries_experiment.loc[time, "Chatly"] = discharge_at_Chatly[0]

     # Track discharge at station location Kerki
    discharge_at_Kerki = experiment.get_value_at_coords(
        "discharge", lat=[Kerki_station_latitude], lon=[Kerki_station_longitude]
    )
    time = experiment.time_as_isostr
    Stations_timeseries_experiment.loc[time, "Kerki"] = discharge_at_Kerki[0]

    # Track discharge at station Tyumen
    discharge_at_Tyumen = experiment.get_value_at_coords(
        "discharge", lat=[Tyumen_station_latitude], lon=[Tyumen_station_longitude]
    )
    time = experiment.time_as_isostr
    Stations_timeseries_experiment.loc[time, "Tyumen"] = discharge_at_Tyumen[0]

    # Track discharge at station location Karalinsk
    discharge_at_Kazalinsk = experiment.get_value_at_coords(
        "discharge", lat=[Kazalinsk_station_latitude], lon=[Kazalinsk_station_longitude]
    )
    time = experiment.time_as_isostr
    Stations_timeseries_experiment.loc[time, "Kazalinsk"] = discharge_at_Kazalinsk[0]


    g.value += 1

print("Model run finished!")

In [ ]:
Stations_timeseries_experiment.plot(label = Stations_timeseries_experiment.columns)

In [ ]:
plt.plot(Stations_timeseries_experiment["Kerki"])
plt.plot(Stations_timeseries["Kerki"])

In [ ]:
import sys

In [ ]:
print(sys.path)

In [ ]:
import imagemean
print(imagemean.__file__)

In [ ]:
import sys
print(sys.path)  # check dat .../PCR-GLOBWB_model/model erin staat



